# Case: Previsão do preço de imóveis no Rio de Janeiro utilizando regressão

## Contexto
O mercado imobiliário é um importante setor da economia brasileira, e a previsão do preço de imóveis é uma questão relevante tanto para compradores quanto para vendedores. Neste caso, o objetivo é desenvolver um modelo de regressão para prever o preço de imóveis na cidade do Rio de Janeiro, utilizando dados públicos disponíveis.

## Base de dados
O Inside Airbnb (http://insideairbnb.com/get-the-data.html) oferece dados sobre anúncios de acomodações no Airbnb, incluindo informações sobre localização, tipo de imóvel, número de quartos e preços. É possível utilizar esses dados para modelar o preço dos imóveis no Rio de Janeiro, filtrando as informações relevantes para o contexto local.

## Aplicação de Ciência de Dados

### 1. Coleta e limpeza de dados
Baixe o arquivo "listings.csv" referente à cidade do Rio de Janeiro no Inside Airbnb. Em seguida, realize a limpeza dos dados removendo entradas duplicadas, tratando valores ausentes e convertendo variáveis categóricas em numéricas.
 - bronze: dado coletado em formato legivel (.parquet)
 - silver: dado filtrado ou convertido em formato (string -> date)
 - gold: dado do imóvel (id, preço, quantidade de quartos)

### 2. Análise exploratória de dados (EDA)
A EDA é realizada para identificar padrões, tendências e correlações entre as variáveis. Por exemplo, verificar a distribuição dos preços dos imóveis, identificar quais bairros possuem os imóveis mais caros e analisar a relação entre o tamanho do imóvel e o preço.

### 3. Feature engineering
Crie novas variáveis relevantes, como a distância até o centro da cidade ou a presença de comércio e serviços nas proximidades. Essas variáveis podem ajudar a melhorar o desempenho do modelo de regressão.

### 4. Modelagem e algoritmos
Separe a base de dados em conjuntos de treinamento e teste. Treine diferentes modelos de regressão, como regressão linear, árvores de decisão e regressão de floresta aleatória (Random Forest). Utilize validação cruzada para avaliar o desempenho dos modelos e selecione aquele com a melhor performance.

### 5. Avaliação e interpretação
Avalie o modelo selecionado utilizando métricas de desempenho, como o erro médio quadrático (RMSE) e o coeficiente de determinação (R²). Interprete os resultados, identificando os fatores que mais influenciam o preço dos imóveis e fornecendo insights para os interessados no mercado imobiliário.

Dessa forma, é possível criar um modelo de regressão para prever o preço de imóveis no Rio de Janeiro utilizando dados disponíveis e técnicas de ciência de dados.


#### Carregando os Dados

In [23]:
import pandas as pd
import pyarrow.parquet as pq
import re
from geopy.distance import distance
import matplotlib.pyplot as plt
import math

In [9]:
!pip install geopy


[notice] A new release of pip is available: 23.0.1 -> 23.1
[notice] To update, run: python -m pip install --upgrade pip


In [1]:
pd.set_option('display.max_columns', None)

NameError: name 'pd' is not defined

In [11]:
mount_path = "/workspaces/prediction_house_price_from_airbnb/Data"

In [27]:
# read the Parquet file into a pyarrow Table

listings = pd.read_parquet(mount_path + "/bronze/listings.parquet")
neighbourhoods = pd.read_parquet(mount_path + "/bronze/neighbourhoods.parquet")
reviews = pd.read_parquet(mount_path + "/bronze/reviews.parquet")
calendar = pd.read_parquet(mount_path + "/bronze/calendar.parquet")



In [25]:
def clean_price_column(df):
    """
    Limpa a coluna 'price' do DataFrame 'df', removendo o símbolo de dólar ($) e a vírgula (,)
    e convertendo o resultado em um valor float. A nova coluna 'price' é adicionada ao DataFrame 'listings'.

    Args:
        df (pandas.DataFrame): O DataFrame a ser limpo.

    Returns:
        pandas.DataFrame: Uma cópia do DataFrame original com a nova coluna 'price' adicionada,
        contendo os valores de preço limpos e convertidos em float.
    """
    listings = df.copy() # cria uma cópia do DataFrame original
    listings['price'] = listings['price'].str.replace('[$,]', '', regex=True).astype(float) # substitui o símbolo de dólar e vírgula e converte para float
    return listings

def extract_bathrooms(df):
    """
    Extrai o número de banheiros da coluna 'bathrooms_text' do DataFrame 'df' e o converte para float,
    salvando o resultado na nova coluna 'bathrooms'.

    Args:
        df (pandas.DataFrame): O DataFrame a ser processado.

    Returns:
        pandas.DataFrame: Uma cópia do DataFrame original com a nova coluna 'bathrooms' adicionada,
        contendo os valores de banheiros extraídos e convertidos em float.
    """
    listings = df.copy() # cria uma cópia do DataFrame original
    listings['bathrooms'] = listings['bathrooms_text'].str.extract(r'(\d+(?:\.\d+)?)?').astype(float) # extrai o número de banheiros e converte para float
    return listings




def calc_distance(row):
    """
    Calcula a distância entre as coordenadas geográficas do imóvel representado pela linha 'row' e as coordenadas do centro do Rio de Janeiro.
    Retorna o valor da distância em quilômetros.

    Args:
        row (pandas.Series): Uma linha do DataFrame 'listings' contendo informações sobre um imóvel.

    Returns:
        float: A distância em quilômetros entre o imóvel e o centro do Rio de Janeiro.
    """
    # Coordenadas geográficas do centro do Rio de Janeiro
    center_lat = -22.908333
    center_lon = -43.196388

    # Coordenadas geográficas do imóvel
    lat = row['latitude']
    lon = row['longitude']

    # Conversão de graus para radianos
    lat_rad = math.radians(lat)
    lon_rad = math.radians(lon)
    center_lat_rad = math.radians(center_lat)
    center_lon_rad = math.radians(center_lon)

    # Cálculo da distância entre as coordenadas geográficas
    delta_lat = center_lat_rad - lat_rad
    delta_lon = center_lon_rad - lon_rad
    a = math.sin(delta_lat/2)**2 + math.cos(lat_rad) * math.cos(center_lat_rad) * math.sin(delta_lon/2)**2
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1-a))
    distance = 6371 * c # raio médio da Terra em quilômetros

    return distance



In [28]:
listings=clean_price_column(listings)
listings=extract_bathrooms(listings)
listings['distance'] = listings.apply(calc_distance, axis=1)

In [ ]:
listing.to_parquet('/workspaces/prediction_house_price_from_airbnb/Data/silver/listings_v01.parquet')